## ある範囲領域の外にいる水分子を消すためのR

これは取りあえず、なんとか動くところまでこぎつけたゴミ関数です。
もしある領域よりも外にある水分子を削除したいと思った方、そして１分弱の計算時間も待てる方は、ご利用ください。
もっと賢い方法はきっと有ります。ただし、自分の用途的に、x軸に関しての制限しか導入していません。拡張は容易いので、自力でやってください。

2018.09.26 made by Kohei Noda (TB Lab., Department of Physics, Nagoya Uniersity)

### 今回の処理の流れ
bio3Dでは原子の選択が若干めんどくさいしたがって次の処理をする。
1. 領域からはみ出ている水分子のresidue numberを把握する。
2. それ以外の分子を選択する。
3. 出力する。

In [238]:
# パッケージの読み込み
library(bio3d) 
library(dplyr)

# pdbの読み込み
pdb <- read.pdb("./system_eq2.pdb")

### resnoを少しアレンジする

In [249]:
# resno が９９９９以上で１から再びカウントされてしまうので足してやるぜ
pdb.atom <- pdb$atom
pdb.boundary <- filter(pdb.atom, pdb.atom[,"resno"] == 9999)

# 境目のeleno をgetしてやるぜ
boundary <- NULL
loopmax <- nrow(pdb.boundary)
for(i in 1:loopmax){
    if(nrow(pdb.boundary) == 0){break}
    boundary <- c(boundary,max(pdb.boundary[,"eleno"]))
    pdb.boundary <- filter(pdb.boundary, pdb.boundary[,"eleno"] < boundary[i] - 9999)
}
boundary.ope <- c(1, rev(boundary), max(pdb.atom[,"eleno"]))
boundary.ope[3]

# 足してやるぜ
for(i in 1:(length(boundary.ope)-1)){
    pdb.atom[boundary.ope[i] : boundary.ope[i+1],"resno"] <- pdb.atom[boundary.ope[i]:boundary.ope[i+1],"resno"] + 10000*(i-1)
}


[1] 54924

## 水分子を選択するぜ

In [250]:
xmin <- -30
xmax <- 30

pdb.atom.select <- filter(pdb.atom, pdb.atom[,"x"] > xmax | pdb.atom[,"x"] < xmin)
pdb.atom.select <- filter(pdb.atom.select, pdb.atom.select[,"resid"] == "WAT")
list.resno <- data.frame("list.resno"=unique(pdb.atom.select[,"resno"]))

D <- NULL
for(i in 1:nrow(list.resno)){
    D <- rbind(D, filter(pdb.atom,pdb.atom[,"resno"]==list.resno[i,]))
}
list.eleno <- data.frame(unique(D$eleno))


## PDBを整えるぜ

In [290]:
D <- NULL
for(i in 1:nrow(list.eleno)){
    A <- atom.select(pdb,eleno=list.eleno[i,])
    D <- combine.select(D,A,operator="+")
}
D <- trim.pdb(pdb,D)
D$atom

 Union of selects
 *  Selected a total of: 2 atoms  *
 Union of selects
 *  Selected a total of: 3 atoms  *
 Union of selects
 *  Selected a total of: 4 atoms  *
 Union of selects
 *  Selected a total of: 5 atoms  *
 Union of selects
 *  Selected a total of: 6 atoms  *
 Union of selects
 *  Selected a total of: 7 atoms  *
 Union of selects
 *  Selected a total of: 8 atoms  *
 Union of selects
 *  Selected a total of: 9 atoms  *
 Union of selects
 *  Selected a total of: 10 atoms  *
 Union of selects
 *  Selected a total of: 11 atoms  *
 Union of selects
 *  Selected a total of: 12 atoms  *
 Union of selects
 *  Selected a total of: 13 atoms  *
 Union of selects
 *  Selected a total of: 14 atoms  *
 Union of selects
 *  Selected a total of: 15 atoms  *
 Union of selects
 *  Selected a total of: 16 atoms  *
 Union of selects
 *  Selected a total of: 17 atoms  *
 Union of selects
 *  Selected a total of: 18 atoms  *
 Union of selects
 *  Selected a total of: 19 atoms  *
 Union of selects


,type,eleno,elety,alt,resid,chain,resno,insert,x,y,z,o,b,segid,elesy,charge
6424,ATOM,6424,O,NA,WAT,NA,457,NA,30.319,46.738,-47.109,1,0,NA,O,NA
6425,ATOM,6425,H1,NA,WAT,NA,457,NA,30.638,47.635,-47.210,1,0,NA,H,NA
6426,ATOM,6426,H2,NA,WAT,NA,457,NA,30.723,46.431,-46.297,1,0,NA,H,NA
6457,ATOM,6457,O,NA,WAT,NA,468,NA,30.140,-40.763,-15.649,1,0,NA,O,NA
6458,ATOM,6458,H1,NA,WAT,NA,468,NA,30.606,-40.847,-14.817,1,0,NA,H,NA
6459,ATOM,6459,H2,NA,WAT,NA,468,NA,30.784,-40.384,-16.247,1,0,NA,H,NA
6589,ATOM,6589,O,NA,WAT,NA,512,NA,30.594,-9.025,16.748,1,0,NA,O,NA
6590,ATOM,6590,H1,NA,WAT,NA,512,NA,30.873,-8.146,17.005,1,0,NA,H,NA
6591,ATOM,6591,H2,NA,WAT,NA,512,NA,31.335,-9.378,16.257,1,0,NA,H,NA
6730,ATOM,6730,O,NA,WAT,NA,559,NA,29.134,30.512,-44.711,1,0,NA,O,NA


## 最後にこれを関数化してやるぜ！

In [2]:
WaterRemover <- function(filename="./",xmin=-20, xmax=20, outputname="output.pdb"){
    if(xmin > xmax){
        message("xmax should be greater than xmin")
        return(201)
    }
    
    # パッケージの読み込み
    library(bio3d) 
    library(dplyr)
    
    # pdbの読み込み
    pdb <- read.pdb(filename)
    
    if(!is.pdb(pdb)){
        message("please check filename! This file is not pdb.")
        return(201)
    }
    
    # resno のアレンジ
    pdb.atom <- pdb$atom
    pdb.boundary <- filter(pdb.atom, pdb.atom[,"resno"] == 9999) #9999以降でまた振り出しに戻るから

    # 境目のeleno をgetしてやるぜ
    boundary <- NULL
    loopmax <- nrow(pdb.boundary)
    for(i in 1:loopmax){
        if(nrow(pdb.boundary) == 0){break}
        boundary <- c(boundary,max(pdb.boundary[,"eleno"]))
        pdb.boundary <- filter(pdb.boundary, pdb.boundary[,"eleno"] < boundary[i] - 9999)
    }
    boundary.ope <- c(1, rev(boundary), max(pdb.atom[,"eleno"]))
    boundary.ope[3]

    # 足してやるぜ
    for(i in 1:(length(boundary.ope)-1)){
        pdb.atom[boundary.ope[i] : boundary.ope[i+1],"resno"] <- pdb.atom[boundary.ope[i]:boundary.ope[i+1],"resno"] + 10000*(i-1)
    }
    
    # 水分子を消してやるぜ
    pdb.atom.select <- filter(pdb.atom, pdb.atom[,"x"] > xmax | pdb.atom[,"x"] < xmin)
    pdb.atom.select <- filter(pdb.atom.select, pdb.atom.select[,"resid"] == "WAT")
    list.resno <- data.frame("list.resno"=unique(pdb.atom.select[,"resno"]))

    D <- NULL
    for(i in 1:nrow(list.resno)){
        D <- rbind(D, filter(pdb.atom,pdb.atom[,"resno"]==list.resno[i,]))
    }
    list.eleno <- data.frame(unique(D$eleno))

    #pdbを整えてやるぜ
    D <- NULL
    for(i in 1:nrow(list.eleno)){
        A <- atom.select(pdb,eleno=list.eleno[i,],inverse=TRUE)
        D <- combine.select(D,A,operator="&")
    }

    D <- trim.pdb(pdb,D)
       
    write.pdb(D, outputname)
}
    

In [ ]:
WaterRemover(filename="./system_eq2.pdb",xmin=-29.4,xmax=29.4)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

